# Multiscale GNN Network

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Introduction

When solving the incompressible Navier-Stokes equations using projection method (or fractional step method), the projection step involves solving the large-scale Pressure Poisson Equation (PPE), which is typically the most computationally expensive and time-consuming step in the entire calculation process. A machine learning-based approach for solving the PPE problem is proposed, and a novel multi-scale Graph Neural Network (GNN) embedded solver is designed to accelerate the numerical solution of the incompressible Navier-Stokes equations. By replacing the traditional iterative solver for solving the PPE, the multi-scale GNN is seamlessly integrated into the numerical solution framework of the incompressible Navier-Stokes equations. In the multi-scale GNN framework, the original high-resolution graph corresponds to the discretized grid of the solution domain, graphs of the same resolution are connected through graph convolution operations, and graphs of different resolutions are connected through up-sampling and down-sampling operations. The well-trained multi-scale GNN serves as a universal PPE solver for a certain class of flow problems.

## Model framework

The model framework is as shown in the following diagram:

![MultiscaleGNN](images/MultiscaleGNN.png)

where

a. Numerical solution framework for the incompressible Navier-Stokes equations embedded with an ML-block (second-order explicit-implicit temporal discretization scheme)；

b. ML-block (multi-scale GNN), $\mathcal{G}^{1h}$ is the original high-resolution graph, $\mathcal{G}^{2h}$, $\mathcal{G}^{3h}$ and  $\mathcal{G}^{4h}$ are the low-resolution graph of level-2, level-3 and level-4, respectively, and the number represents the number of neurons in the corresponding layer.

## Model training

Import code packs

In [ ]:
import os
import argparse

import numpy as np
import scipy.io as sio
import mindspore
from mindspore import nn, ops, context, Tensor, COOTensor
import mindspore.dataset as ds

from src.models import MultiScaleGNN, MultiScaleGNNStructure
from src.datasets import read_training_data, read_test_data
from src.datasets import second_order_derivative_matix2d
from src.visualization import losses_curve, contourf_comparison

The setting of optional parameters, the setting of training hyperparameters and the construction of the model

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--grid_type', type=str, default='unstructure')
parser.add_argument('--activ_fun', type=str, default='swish')
parser.add_argument('--device', type=str, default='CPU')
parser.add_argument('--batch_size', type=int, default=5)
parser.add_argument('--lambda_p', type=int, default=1)
parser.add_argument('--lambda_eq', type=int, default=1)
args = parser.parse_args()

grid_type = args.grid_type
activ_fun = args.activ_fun
device = args.device
batch_size = args.batch_size
lambda_p = args.lambda_p
lambda_eq = args.lambda_eq

context.set_context(device_target=device)

nx = 512
ny = 512
dx = 2.0 * np.pi / nx
dy = 2.0 * np.pi / ny

t_train = np.arange(0.1, 5.1, 0.1)
t_test = [2.0, 4.0, 6.0, 8.0]

div_u_star_train, p_train = read_training_data(t_train, dx, dy)

print('divUstar_train shape :', div_u_star_train.shape)
print('p_train shape :', p_train.shape)
print('divUstar_train max :', np.abs(div_u_star_train).max())
print('p_train max :', np.abs(p_train).max())

divUstar_test, p_test = read_test_data(t_test, dx, dy)
print('divUstar_test shape :', divUstar_test.shape)
print('p_test shape :', p_test.shape)
print('divUstar_test max :', np.abs(divUstar_test).max())
print('p_test max :', np.abs(p_test).max())

batch = div_u_star_train.shape[0]//batch_size

A0 = second_order_derivative_matix2d(nx, ny, dx, dy)

save_dir = f'./Savers/{grid_type}/{activ_fun}_lambda_p{lambda_p}lambda_eq{lambda_eq}'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

A0_tensor = COOTensor(Tensor(np.stack((A0.row, A0.col), axis=1)), \
                      Tensor(A0.data, dtype=mindspore.float32), A0.shape)

in_channels = 1
out_channels = 1
if activ_fun == 'swish':
    activation = nn.SiLU()
if activ_fun == 'elu':
    activation = nn.ELU()
if activ_fun == 'gelu':
    activation = nn.GELU()

if grid_type == 'structure':
    model = MultiScaleGNNStructure(in_channels, out_channels, activation)
if grid_type == 'unstructure':
    model = MultiScaleGNN(in_channels, out_channels, activation, A0)

train_dataset = ds.NumpySlicesDataset(data=(div_u_star_train, p_train), shuffle=True)
train_dataset = train_dataset.batch(batch_size=batch_size)

learning_rate = nn.piecewise_constant_lr([500*batch, 1000*batch, 1500*batch, 2000*batch], [1e-3, 1e-4, 1e-5, 1e-6])
optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=learning_rate, weight_decay=1e-6)

The main part of the training code includes the calculation of loss function, the forward calculation process and the training process

In [ ]:
def get_weights_norm(model):
    weights_norm = 0.0
    for name, param in model.parameters_and_names():
        if 'weight' in name:
            weights_norm += param.norm()**2
    return weights_norm

def calculate_loss_p(p_hat, p):
    loss_p = ops.mean((p_hat - p)**2)
    return loss_p

def calculate_loss_eq(p_hat, b):
    p_hat = p_hat * 0.01
    p_hat = ops.cat((p_hat[:, -1:], p_hat, p_hat[:, 0:1]), axis=1)
    p_hat = ops.cat((p_hat[:, :, -1:], p_hat, p_hat[:, :, 0:1]), axis=2)
    b_hat = (p_hat[:, 0:-2, 1:-1] - 2 * p_hat[:, 1:-1, 1:-1] + p_hat[:, 2:, 1:-1]) / dx**2 + \
            (p_hat[:, 1:-1, 0:-2] - 2 * p_hat[:, 1:-1, 1:-1] + p_hat[:, 1:-1, 2:]) / dy**2
    loss_eq = ops.mean((b_hat - b)**2)
    return loss_eq

def forward_fn(b, p):
    if grid_type == 'unstructure':
        inputs = b.reshape([-1, 1])
        p_hat = model(inputs)
        p_hat = p_hat.reshape([batch_size, nx, ny])
    if grid_type == 'structure':
        inputs = b.unsqueeze(1)
        p_hat = model(inputs)
        p_hat = p_hat.squeeze(1)
    loss_p = calculate_loss_p(p_hat, p)
    loss_eq = calculate_loss_eq(p_hat, b)
    loss = lambda_p * loss_p + lambda_eq * loss_eq
    return loss, loss_p, loss_eq

grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

def train_step(b, p):
    (loss, loss_p, loss_eq), grads = grad_fn(b, p)
    optimizer(grads)
    return loss, loss_p, loss_eq

def run_train(model, num_epochs, restore=False):
    if not os.path.exists(save_dir+'/checkpoint/'):
        os.makedirs(save_dir+'/checkpoint/')
    if not os.path.exists(save_dir+'/Figures/'):
        os.makedirs(save_dir+'/Figures/')
    if restore:
        loss_all = sio.loadmat(save_dir+'/loss_all.mat')['loss_all'].tolist()
        param_dict = mindspore.load_checkpoint(save_dir+'/checkpoint/model.ckpt')
        param_not_load, _ = mindspore.load_param_into_net(model, param_dict)
        print(param_not_load)
        model.set_train()

    else:
        loss_all = []

    print('Training......')
    model.set_train()
    weights_norm = get_weights_norm(model)
    for epoch in range(1, num_epochs+1):
        loss_p_it, loss_eq_it = [], []
        for b, p in train_dataset.create_tuple_iterator():
            _, loss_p, loss_eq = train_step(b, p)

            loss_p_it.append(loss_p.asnumpy())
            loss_eq_it.append(loss_eq.asnumpy())

            print('Epoch: %d, weights_norm: %.3e, loss_p: %.3e, loss_eq: %.3e' % (epoch, weights_norm.asnumpy(), np.mean(loss_p_it), np.mean(loss_eq_it)))
        weights_norm = get_weights_norm(model)
        print('***********************************************************')
        loss_all.append([weights_norm.asnumpy(), np.mean(loss_p_it), np.mean(loss_eq_it)])

        if epoch % 1 == 0:
            mindspore.save_checkpoint(model, save_dir+'/checkpoint/model.ckpt')
            sio.savemat(save_dir+'/loss_all.mat', {'loss_all':np.array(loss_all)})

        if epoch % 10 == 0:
            losses_curve(np.array(loss_all), save_dir+'/Figures/')

        if epoch % 100 == 0:
            model.set_train(False)
            if grid_type == 'unstructure':
                inputs = Tensor(divUstar_test).reshape([-1, 1])
                p_hat = model(inputs)
                p_hat = p_hat.reshape([4, nx, ny]).asnumpy()
            if grid_type == 'structure':
                inputs = Tensor(divUstar_test).unsqueeze(1)
                p_hat = model(inputs)
                p_hat = p_hat.squeeze(1).asnumpy()
            for k in range(4):
                contourf_comparison(p_test[k], p_hat[k], save_dir+f'/Figures/{k}')

            model.set_train()

In [ ]:
run_train(model, num_epochs=2000, restore=False)

## Model test

Test parameter configuration

In [ ]:
grid_type = 'structure'
activ_fun = 'swish'
device = 'CPU'
lambda_p = 50
lambda_eq = 1
plot_figure = 1

context.set_context(device_target=device)

nx = 512
ny = 512
dx = 2.0*np.pi/nx
dy = 2.0*np.pi/ny

t_test = [2.0, 4.0, 6.0, 8.0, 10.0]

divUstar_test, p_test = read_test_data(t_test, dx, dy)
print('divUstar_test shape :', divUstar_test.shape)
print('p_test shape :', p_test.shape)
print('divUstar_test max :', np.abs(divUstar_test).max())
print('p_test max :', np.abs(p_test).max())

A0 = second_order_derivative_matix2d(nx, ny, dx, dy)

save_dir = f'./Savers/{grid_type}/{activ_fun}_lambda_p{lambda_p}lambda_eq{lambda_eq}'

A0_tensor = COOTensor(Tensor(np.stack((A0.row, A0.col), axis=1)), \
                      Tensor(A0.data, dtype=mindspore.float32), A0.shape)

in_channels = 1
out_channels = 1
if activ_fun == 'swish':
    activation = nn.SiLU()
if activ_fun == 'elu':
    activation = nn.ELU()
if activ_fun == 'gelu':
    activation = nn.GELU()

if grid_type == 'structure':
    model = MultiScaleGNNStructure(in_channels, out_channels, activation)
if grid_type == 'unstructure':
    model = MultiScaleGNN(in_channels, out_channels, activation, A0)

Define the main function for test

In [ ]:
def run_test(model):
    loss_all = sio.loadmat(save_dir+'/loss_all.mat')['loss_all'].tolist()
    param_dict = mindspore.load_checkpoint(save_dir+'/checkpoint/model.ckpt')
    param_not_load, _ = mindspore.load_param_into_net(model, param_dict)
    print(param_not_load)
    model.set_train(False)

    losses_curve(np.array(loss_all), save_dir+'/Figures/')

    p_hat = []
    for it in range(divUstar_test.shape[0]//5):
        if grid_type == 'unstructure':
            b = Tensor(divUstar_test[it*5:(it+1)*5]).reshape([-1, 1])
            p = model(b)
            p = p.reshape([5, nx, ny]).asnumpy()
            p_hat.append(p)
        if grid_type == 'structure':
            b = Tensor(divUstar_test[it*5:(it+1)*5]).unsqueeze(1)
            p = model(b)
            p = p.squeeze(1).asnumpy()
            p_hat.append(p)

    p_hat = np.concatenate(p_hat, axis=0)

    print('####################################')
    MAE = np.mean((p_hat-p_test)**2, axis=(1, 2))
    print('mean absolute error:')
    print(MAE)
    print('average mean absolute error:')
    print(MAE.mean())

    print('####################################')
    RL2E = np.linalg.norm(p_hat-p_test, axis=(1, 2))**2/np.linalg.norm(p_test, axis=(1, 2))**2
    print('relative L2 error:')
    print(RL2E)
    print('average relative L2 error:')
    print(RL2E.mean())

    if plot_figure:
        for k in range(divUstar_test.shape[0]):
            contourf_comparison(p_test[k], p_hat[k], save_dir+f'/Figures/{k}')

Testing

In [ ]:
run_test(model)